In [24]:
! pip3 install tensorflow

In [29]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
from tensorflow.keras.models import Sequential, load_model

# Load the dataset
data = pd.read_csv('new.csv')
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.set_index('timestamp', inplace=True)

data.fillna(data.mean(), inplace=True)
# Calculate the first quartile (Q1) and third quartile (Q3)
Q1 = data['visits'].quantile(0.25)
Q3 = data['visits'].quantile(0.75)

# Calculate the interquartile range (IQR)
IQR = Q3 - Q1

# Define lower and upper bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers = data[(data['visits'] < lower_bound) | (data['visits'] > upper_bound)]

# Remove outliers
data_no_outliers = data[~((data['visits'] < lower_bound) | (data['visits'] > upper_bound))]

# Normalize the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data_no_outliers)

# Define a function to prepare data for LSTM model
def prepare_data(data, n_steps):
    X, y = [], []
    for i in range(len(data)):
        end_ix = i + n_steps
        if end_ix > len(data)-1:
            break
        seq_x, seq_y = data[i:end_ix], data[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Define the number of time steps
n_steps = 30

# Prepare data for LSTM model
X, y = prepare_data(scaled_data, n_steps)

# Split data into training and validation sets (using the entire dataset for training)
train_size = len(X)
X_train, y_train = X[:train_size], y[:train_size]


# Step 2: Model Selection
# Define LSTM model
model = Sequential([
    LSTM(50, activation='relu', input_shape=(n_steps, X_train.shape[2])),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Step 4: Training and Validation
# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

# Step 6: Forecasting
# Forecasting for the next 50 days
forecast = []
current_batch = X[-1].reshape((1, n_steps, X_train.shape[2]))
for i in range(50):
    current_pred = model.predict(current_batch)[0]
    forecast.append(current_pred)
    current_batch = np.append(current_batch[:, 1:, :], [[current_pred]], axis=1)

# Inverse scaling to get the actual forecasted values
forecast = scaler.inverse_transform(forecast)

# Display the forecasted values
print("Forecasted attacks for the next 50 days:")
print(np.floor(forecast))
final_forecast = np.floor(forecast)
# Step 5: Model Evaluation
# Predictions on training set
y_pred = model.predict(X_train)

# Inverse transform the predicted and actual values to their original scale
y_pred_inv = scaler.inverse_transform(y_pred)
y_train_inv = scaler.inverse_transform(y_train)

# Calculate RMSE
rmse = math.sqrt(mean_squared_error(y_train_inv, y_pred_inv))
print("Root Mean Squared Error (RMSE) on training set:", rmse)

# Calculate MAE
mae = mean_absolute_error(y_train_inv, y_pred_inv)
print("Mean Absolute Error (MAE) on training set:", mae)


Epoch 1/100


C:\Users\George Welson\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.1765
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0423
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0382
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0360
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0361
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0359
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0400
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0350
Epoch 9/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0347
Epoch 10/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0389
Epoch 11/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0392
Epoch 12/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0381
Epoch 13/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.0341
Epoch 14/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0344
Epoch 15/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0368
Epoch 16/100
15/

In [30]:
model.save('my_model.h5')

In [33]:
loaded_model = load_model('E:\dell_hacktrick\my_model.h5')

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\George Welson\AppData\Local\Temp\ipykernel_32772\2168681073.py:1: SyntaxWarning: invalid escape sequence '\d'
  loaded_model = load_model('E:\dell_hacktrick\my_model.h5')
C:\Users\George Welson\AppData\Local\Temp\ipykernel_32772\2168681073.py:1: SyntaxWarning: invalid escape sequence '\d'
  loaded_model = load_model('E:\dell_hacktrick\my_model.h5')


ValueError: Could not interpret initializer identifier: {'module': 'keras.initializers', 'class_name': 'Orthogonal', 'config': {'gain': 1.0, 'seed': None}, 'registered_name': None}

In [ ]:
# Compile the model
loaded_model.compile(optimizer='adam', loss='mse')

# Step 4: Training and Validation
# Train the model
loaded_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1)

# Step 6: Forecasting
# Forecasting for the next 50 days
forecast = []
current_batch = X[-1].reshape((1, n_steps, X_train.shape[2]))
for i in range(50):
    current_pred = model.predict(current_batch)[0]
    forecast.append(current_pred)
    current_batch = np.append(current_batch[:, 1:, :], [[current_pred]], axis=1)

# Inverse scaling to get the actual forecasted values
forecast = scaler.inverse_transform(forecast)

# Display the forecasted values
print("Forecasted attacks for the next 50 days:")
print(np.floor(forecast))
final_forecast = np.floor(forecast)
# Step 5: Model Evaluation
# Predictions on training set
y_pred = model.predict(X_train)

# Inverse transform the predicted and actual values to their original scale
y_pred_inv = scaler.inverse_transform(y_pred)
y_train_inv = scaler.inverse_transform(y_train)

# Calculate RMSE
rmse = math.sqrt(mean_squared_error(y_train_inv, y_pred_inv))
print("Root Mean Squared Error (RMSE) on training set:", rmse)

# Calculate MAE
mae = mean_absolute_error(y_train_inv, y_pred_inv)
print("Mean Absolute Error (MAE) on training set:", mae)
